# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

In [116]:
# load messages dataset
messages = pd.read_csv("messages.csv")
messages.head()
# messages.shape
# messages.isnull().sum()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [117]:
# load categories dataset
categories = pd.read_csv("categories.csv")
categories.head()
# categories.shape
# categories.isnull().sum()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [118]:
# merge datasets
df = messages.merge(categories, on='id')
df.head()
# df.shape
# df.isnull().sum()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [119]:
# create a dataframe of the 36 individual category columns

categories = df['categories'].str.split(';', expand=True)
 
categories.head()
# categories.shape

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [120]:
# select the first row of the categories dataframe
row = df.loc[0, 'categories']

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = re.sub("[-, \d]",'', row).split(';')
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [121]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [122]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].astype(str).apply(lambda x: x[-1])
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [123]:
# drop the original categories column from `df`
df.drop(['categories'], axis = 1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [124]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis = 1)
df.head()
# df.shape

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6A. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [125]:
# check number of duplicates
df.duplicated().sum()

170

In [126]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [127]:
# check number of duplicates
df.duplicated().sum()

0

### 6B. Further Cleaning

Find if all values are 1 and 0 or not.

In [128]:
for col in df.columns.tolist()[4:]:
    print('Value counts of category {} is:'.format(col) )
    print(df[col].value_counts())

Value counts of category related is:
1    19906
0     6122
2      188
Name: related, dtype: int64
Value counts of category request is:
0    21742
1     4474
Name: request, dtype: int64
Value counts of category offer is:
0    26098
1      118
Name: offer, dtype: int64
Value counts of category aid_related is:
0    15356
1    10860
Name: aid_related, dtype: int64
Value counts of category medical_help is:
0    24132
1     2084
Name: medical_help, dtype: int64
Value counts of category medical_products is:
0    24903
1     1313
Name: medical_products, dtype: int64
Value counts of category search_and_rescue is:
0    25492
1      724
Name: search_and_rescue, dtype: int64
Value counts of category security is:
0    25745
1      471
Name: security, dtype: int64
Value counts of category military is:
0    25356
1      860
Name: military, dtype: int64
Value counts of category child_alone is:
0    26216
Name: child_alone, dtype: int64
Value counts of category water is:
0    24544
1     1672
Name: wat

You can see that all the values of category "child_alone" are zero. So, remove it.

In [129]:
df.drop(['child_alone'], axis = 1, inplace=True, errors='ignore')

The "related" category has also 2 values. Let's change them to 1 as it is the majority.

In [130]:
df['related'] = df['related'].map(lambda x: 1 if x==2 else x)
df['related'].value_counts()

1    20094
0     6122
Name: related, dtype: int64

The "message" column has https links. Let's convert them to urlplaceholder

In [131]:

def change_url(text):
    """ Function to convert url linke (https:...) to a a string (urlplaceholder)
    Input: text string
    Output: text url link changed to "urlplaceholder"
    """
    
    # regular expression to detect a url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_regex2 = 'http.*(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    detected_urls2 = re.findall(url_regex2, text)
    detected_urls.extend(detected_urls2)
    

    
    # replace each url in text string with placeholder
    if detected_urls != []:
        for url in detected_urls:
            text = text.replace(url, "urlplaceholder")
            return text
    else:
        return text
    
    

In [133]:
## iterated 10 time only because some message have several spaces between
## "http" and the rest of the url link (check later for a better solution)
for i in range(1,10):
    df['message'] = df['message'].map(change_url)

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [134]:
# check if there is any http in the message
n = 0
for message in df['message']:
    if 'http' in message:
        n += 1
        print(n,')', message)


In [66]:

engine = create_engine('sqlite:///DisasterResponseDatabase.db')
df.to_sql('DisasterResponseTable', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

# 9. Refractoring

In [10]:
def load_data(messages_filepath, categories_filepath):
    """ Function to load messages and categorie data sets
    Input: messages and categorie csv files
    Output:  merged dataframe from messages and categorie datasets
    """

    messages = pd.read_csv(messages_filepath)
    categories = pd.read_csv(categories_filepath)
    df = messages.merge(categories, on='id') # merge datasets
    # messages.shape
    # messages.isnull().sum()
    # categories.shape
    # categories.isnull().sum()

    return df



def change_url(text):
    """ Function to convert url linke (https:...) to a a string (urlplaceholder)
    Input: text string
    Output: text url link changed to "urlplaceholder"
    """
    
    # regular expression to detect a url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_regex2 = 'http.*(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    detected_urls2 = re.findall(url_regex2, text)
    detected_urls.extend(detected_urls2)
    

    
    # replace each url in text string with placeholder
    if detected_urls != []:
        for url in detected_urls:
            text = text.replace(url, "urlplaceholder")
            return text
    else:
        return text
    


def clean_data(df):
    """ Function to clean dataframe 
    Input: df dataframe
    Output: Clean dataframe df with seperated categories, 
            numeric column values, and removed duplicates
    """

    ## 1. Split `categories` into separate category columns
    # create a dataframe of the 36 individual category columns
    categories = df['categories'].str.split(';', expand=True)
    # select the first row of the categories dataframe
    row = df.loc[0, 'categories']
    # use this row to extract a list of new column names for categories.
    category_colnames = re.sub("[-, \d]",'', row).split(';')
    # rename the columns of `categories`
    categories.columns = category_colnames
#     print('... Splited `categories` into separate category columns ...')

    ## 2. Convert category values to just numbers 0 or 1
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].astype(str).apply(lambda x: x[-1])
        # convert column from string to numeric
        categories[column] = pd.to_numeric(categories[column])
#     print('... Converted category values to just numbers 0 or 1 ...')
    
    ## 3. Replace `categories` column in `df` with new category columns:
            # - Drop the categories column from the df dataframe since it is no longer needed.
            # - Concatenate df and categories data frames.
    df.drop(['categories'], axis = 1, inplace=True)
    df = pd.concat([df, categories], axis = 1)

    ## 4. Remove duplicates.
        # - Check how many duplicates are in this dataset.
        # - Drop the duplicates.
        # - Confirm duplicates were removed.
    df.drop_duplicates(inplace=True)
    
    # all the values of category "child_alone" are zero
    df.drop(['child_alone'], axis = 1, inplace=True, errors='ignore')
    
    # The "related" category has also 2 values. Let's change them to 1 as it is the majority.
    df['related'] = df['related'].map(lambda x: 1 if x==2 else x)
    
    
    ## change url links (http.....) with string "urlplaceholder"
    ## iterated 10 time only because some message have several spaces between
    ## "http" and the rest of the url link (check later for a better solution)
    for i in range(1,10):
        df['message'] = df['message'].map(change_url)

    return df



def save_data(df, database_filename):
    """ Function to save the clean dataset into an sqlite database
    Input: cleaned dataframe df
    Output: DisasterResponse Database and DisasterResponse Table
    """
    
    con = 'sqlite:///'+ database_filepath
    engine = create_engine(con)
    return df.to_sql('DisasterResponseTable', engine, index=False, if_exists='replace')



messages_filepath = "messages.csv"
categories_filepath = "categories.csv"

print('... Loading the data ...')
df = load_data(messages_filepath, categories_filepath)

print('... Cleaning the data...')
df = clean_data(df)

database_filepath = 'DisasterResponse.db'
print('... Saving data:    DATABASE: {}'.format(database_filepath))
save_data(df, database_filepath)


print('\n... Cleaned data and saved to database! - Finished')


... Loading the data ...
... Cleaning the data...
... Saving data:    DATABASE: DisasterResponse.db

... Cleaned data and saved to database!
